# 🔌 MCP Integration with Strands Agents

This notebook demonstrates how to integrate Model Context Protocol (MCP) servers with Strands Agents, enabling access to external systems and specialized tools.

## 📋 What You'll Learn

- Understanding Model Context Protocol (MCP) fundamentals
- Connecting to MCP servers from Strands Agents
- AWS Documentation MCP server integration
- Building MCP-enabled agents for external system access

## 🎯 Prerequisites

- Complete notebooks 01-06 for foundational knowledge
- Understanding of agent tools and multi-agent systems
- Basic knowledge of external APIs and protocols

## 📦 Installation and Setup

In [1]:
!pip install -r requirements.txt

## 🔧 Configuration and Imports

In [2]:
import json
import os
from pathlib import Path
import logging

# Configure environment
os.environ.setdefault("AWS_REGION", "us-west-2")
logging.getLogger("strands").setLevel(logging.WARNING)

# Optional: Set MEM0 API key for memory features (get your free key from https://mem0.ai)
MEM0_API_KEY = os.getenv("MEM0_API_KEY")
if MEM0_API_KEY:
    os.environ["MEM0_API_KEY"] = MEM0_API_KEY

print("🔌 MCP integration setup complete!")

🔌 MCP integration setup complete!


## 🌐 Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI agents to connect to external systems and tools:

### Key Benefits:
- **External System Access**: Connect to databases, APIs, and services
- **Specialized Tools**: Access domain-specific functionality
- **Real-time Data**: Get up-to-date information from live systems
- **Standardized Interface**: Consistent way to connect to different services

### Common MCP Servers:
- **AWS Documentation**: Search and access AWS docs
- **Database Connectors**: Query databases directly
- **File Systems**: Access local and remote files
- **Web APIs**: Connect to REST and GraphQL services

## 📋 MCP Configuration Check

Let's check for MCP configuration and handle missing files gracefully:

## 🔧 MCP Setup Guide

### To Enable Full MCP Functionality:

1. **Install MCP Server**:
   ```bash
   uvx awslabs.aws-documentation-mcp-server@latest
   ```

2. **Create mcp.json Configuration**:
   ```json
   {
     "mcpServers": {
       "awslabs.aws-documentation-mcp-server": {
         "command": "uvx",
         "args": ["awslabs.aws-documentation-mcp-server@latest"],
         "env": {
           "FASTMCP_LOG_LEVEL": "ERROR",
           "AWS_DOCUMENTATION_PARTITION": "aws"
         },
         "disabled": false,
         "autoApprove": []
       }
     }
   }
   ```

3. **Place Configuration File**:
   - Save as `mcp.json` in the scripts directory
   - Or in the same directory as this notebook

4. **Restart Notebook**:
   - Restart the kernel and run all cells
   - The agent will automatically detect and use MCP

In [3]:
def check_mcp_config():
    """Check for MCP configuration file"""
    # Try multiple possible locations
    possible_paths = [
        Path.cwd() / "mcp.json",
        Path.cwd() / "scripts" / "mcp.json",
        Path("../scripts/mcp.json"),
    ]
    
    for mcp_path in possible_paths:
        if mcp_path.exists():
            try:
                with open(mcp_path, 'r') as f:
                    config = json.load(f)
                print(f"✅ Found MCP config at: {mcp_path}")
                servers = config.get("mcpServers", {})
                print(f"🔌 Available servers: {list(servers.keys())}")
                return config
            except Exception as e:
                print(f"⚠️ Error reading {mcp_path}: {e}")
    
    print("❌ No MCP configuration found")
    print("💡 This is okay - we'll show you how MCP works conceptually")
    return None

# Check for MCP configuration
mcp_config = check_mcp_config()

✅ Found MCP config at: ../scripts/mcp.json
🔌 Available servers: ['awslabs.aws-documentation-mcp-server', 'fetch', 'strands']


## 🏗️ MCP Integration Concepts

Even without a live MCP server, let's understand how MCP integration works:

In [4]:
# Import MCP components (with error handling)
try:
    from mcp import stdio_client, StdioServerParameters
    from strands import Agent
    from strands.models import BedrockModel
    from strands.tools.mcp import MCPClient
    
    mcp_available = True
    print("✅ MCP components imported successfully")
except ImportError as e:
    mcp_available = False
    print(f"⚠️ MCP components not available: {e}")
    print("💡 This is okay - we'll show the concepts")

# Show MCP integration pattern
print("\n🔧 MCP Integration Pattern:")
print("""
1. Configure MCP Server (mcp.json)
2. Create MCP Client
3. Get Tools from MCP Server
4. Create Agent with MCP Tools
5. Use Agent with External Capabilities
""")

✅ MCP components imported successfully

🔧 MCP Integration Pattern:

1. Configure MCP Server (mcp.json)
2. Create MCP Client
3. Get Tools from MCP Server
4. Create Agent with MCP Tools
5. Use Agent with External Capabilities



## 🤖 Conceptual MCP Agent

Let's show how an MCP-enabled agent would work:

In [5]:
class ConceptualMCPAgent:
    """Demonstrates MCP integration concepts"""
    
    def __init__(self):
        self.mcp_available = mcp_available and mcp_config is not None
        
        if self.mcp_available:
            print("🤖 MCP Agent: Full functionality available")
            self._setup_real_mcp()
        else:
            print("🤖 MCP Agent: Demo mode (showing concepts)")
            self._setup_demo_mode()
    
    def _setup_real_mcp(self):
        """Setup real MCP integration"""
        try:
            # This would be the real MCP setup
            aws_docs_config = mcp_config["mcpServers"]["awslabs.aws-documentation-mcp-server"]
            
            self.mcp_client = MCPClient(lambda: stdio_client(
                StdioServerParameters(
                    command=aws_docs_config["command"],
                    args=aws_docs_config["args"],
                    env=aws_docs_config.get("env", {})
                )
            ))
            
            self.model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                region_name="us-west-2",
                temperature=0.3,
            )
            
            print("✅ Real MCP client configured")
            
        except Exception as e:
            print(f"⚠️ MCP setup failed: {e}")
            self.mcp_available = False
            self._setup_demo_mode()
    
    def _setup_demo_mode(self):
        """Setup demo mode without real MCP"""
        self.demo_responses = {
            "s3": "Amazon S3 is a scalable object storage service that offers industry-leading durability, availability, and performance.",
            "lambda": "AWS Lambda is a serverless compute service that runs code in response to events and automatically manages compute resources.",
            "ecs": "Amazon ECS is a container orchestration service that supports Docker containers and allows you to run applications on a managed cluster."
        }
        print("✅ Demo mode configured with sample responses")
    
    def query(self, user_input: str) -> str:
        """Process user query (real MCP or demo)"""
        if self.mcp_available:
            return self._real_mcp_query(user_input)
        else:
            return self._demo_query(user_input)
    
    def _real_mcp_query(self, user_input: str) -> str:
        """Real MCP query processing"""
        try:
            with self.mcp_client:
                tools = self.mcp_client.list_tools_sync()
                
                agent = Agent(
                    model=self.model,
                    system_prompt="You are an AWS Documentation expert. Provide concise, practical answers.",
                    tools=tools
                )
                
                return agent(user_input)
        except Exception as e:
            return f"❌ MCP query failed: {e}"
    
    def _demo_query(self, user_input: str) -> str:
        """Demo query processing"""
        user_lower = user_input.lower()
        
        for service, response in self.demo_responses.items():
            if service in user_lower:
                return f"📚 [DEMO] {response}"
        
        return "📚 [DEMO] This would query the AWS Documentation MCP server for real-time information."

# Create the conceptual MCP agent
mcp_agent = ConceptualMCPAgent()

🤖 MCP Agent: Full functionality available
✅ Real MCP client configured


## 🧪 Testing MCP Concepts

Let's test our MCP agent (real or demo mode):

In [ ]:
# Test queries
test_queries = [
    "What is Amazon S3?",
    "How does AWS Lambda work?",
    "Tell me about Amazon ECS",
    "What are EC2 instance types?"
]

print("🔍 Testing MCP Agent with sample queries:")
print("=" * 50)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: {query}")
    response = mcp_agent.query(query)

print("\n✅ MCP concept testing complete!")

🔍 Testing MCP Agent with sample queries:

1. Query: What is Amazon S3?
I'll search for information about Amazon S3 to provide you with a comprehensive overview.
Tool #1: search_documentation
Perfect! Let me get the detailed information about Amazon S3 from the main documentation page.
Tool #2: read_documentation
Based on the AWS documentation, here's what Amazon S3 is:

## Amazon S3 (Simple Storage Service)

**Amazon S3** is an **object storage service** that provides industry-leading scalability, data availability, security, and performance. It's designed to store and protect any amount of data for various use cases.

### Key Characteristics:

- **Object Storage**: Stores files as objects in containers called "buckets"
- **Scalable**: Can handle data from small applications to massive enterprise workloads
- **Highly Available**: Built for 99.999999999% (11 9's) durability
- **Secure**: Multiple layers of security and access controls

### Common Use Cases:

- **Data Lakes** - Central r

## 🚀 MCP Benefits and Use Cases

### Real-World Applications:

1. **Documentation Assistant**:
   - Query AWS docs in real-time
   - Get up-to-date service information
   - Find best practices and examples

2. **Database Integration**:
   - Query databases directly from agents
   - Get real-time data for decisions
   - Update records based on conversations

3. **File System Access**:
   - Read and write files
   - Process documents and data
   - Manage file operations

4. **API Integration**:
   - Connect to REST APIs
   - Fetch external data
   - Integrate with third-party services

### Production Considerations:
- **Security**: Secure MCP server connections and authentication
- **Performance**: Monitor MCP call latency and optimize
- **Reliability**: Implement retry mechanisms and error handling
- **Monitoring**: Log MCP interactions for debugging and analysis

## 📚 Summary

You've learned about:

✅ **MCP fundamentals** and protocol benefits  
✅ **Configuration patterns** for MCP servers  
✅ **Integration concepts** with Strands Agents  
✅ **Real-world applications** and use cases  

### 🔑 Key Takeaways:
- **MCP enables external system access** for agents
- **Configuration is crucial** for successful connections
- **Error handling is essential** for robust applications
- **Multiple servers can be combined** for comprehensive toolsets

### 🚀 Next Steps:
- Set up MCP configuration to enable full functionality
- Explore other MCP servers (database, file system, web APIs)
- Build custom MCP servers for your specific needs
- Integrate MCP with multi-agent systems from notebook 06

### 🎯 Production Deployment:
- Implement proper authentication and security
- Monitor MCP server performance and availability
- Set up logging and error tracking
- Plan for scaling and load balancing

**Congratulations! You've completed the Strands Agents Workshop! 🎉**

You now have the knowledge to build:
- Basic and advanced agents
- Custom tools and integrations
- Multi-agent systems
- External system connections via MCP

**Ready to build amazing AI applications! 🚀**